In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog, local_binary_pattern

In [7]:
DATASET_PATH = "../dataset/augmented"
SAVE_PATH = "../dataset/processed"

os.makedirs(SAVE_PATH, exist_ok=True)
print(os.path.abspath(DATASET_PATH))


CLASSES = []
for d in os.listdir(DATASET_PATH):
    full_path = os.path.join(DATASET_PATH, d)
    if os.path.isdir(full_path) and d != 'unknown':
        CLASSES.append(d)


CLASSES = sorted(CLASSES)
CLASSES

D:\machine_project\MSI_waste_sorting\dataset\augmented


['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [8]:
label_map = {cls: idx for idx, cls in enumerate(CLASSES)}
label_map

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [13]:
IMG_SIZE = (128, 128)

def preprocess(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, IMG_SIZE)
    return img

In [33]:
def extract_hog(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = hog(
        gray,
        orientations=18,
        pixels_per_cell=(16, 16),
        cells_per_block=(3, 3),
        block_norm='L2-Hys'
    )
    return features

def extract_lbp(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=10, range=(0, 10))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist


def extract_color(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8],
                         [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist


In [38]:
FEATURE_TYPE = "HOG"  # Options: "HOG", "LBP", "COLOR"

def extract_features(img):
    if FEATURE_TYPE.upper() == "HOG":
        return extract_hog(img)
    elif FEATURE_TYPE.upper() == "LBP":
        return extract_lbp(img)
    elif FEATURE_TYPE.upper() == "COLOR":
        return extract_color(img)
    else:
        raise ValueError("Unknown FEATURE_TYPE!")


X, y = [], []

for cls in CLASSES:
    cls_path = os.path.join(DATASET_PATH, cls)
    for img_name in tqdm(os.listdir(cls_path), desc=cls):
        img_path = os.path.join(cls_path, img_name)
        img = preprocess(img_path)

        features = np.concatenate([extract_hog(img), extract_color(img), extract_lbp(img)])

        X.append(features)
        y.append(label_map[cls])

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


trash: 100%|██████████| 476/476 [00:03<00:00, 158.45it/s]

X shape: (2856, 6354)
y shape: (2856,)


In [39]:
np.save(os.path.join(SAVE_PATH, "X_hog.npy"), X)
np.save(os.path.join(SAVE_PATH, "y.npy"), y)

In [40]:
print("HOG feature length:", X.shape[1])


HOG feature length: 6354


In [41]:
import numpy as np

# Load HOG features
X_hog = np.load("../dataset/processed/X_hog.npy")
y = np.load("../dataset/processed/y.npy")

print("X_hog shape:", X_hog.shape)
print("y shape:", y.shape)

# Check the first vector
print("First feature vector (HOG) length:", len(X_hog[0]))
print("First label:", y[0])


X_hog shape: (2856, 6354)
y shape: (2856,)
First feature vector (HOG) length: 6354
First label: 0
